<a href="https://colab.research.google.com/github/ad451/Stackoverflow_tag_generator/blob/main/StackOverflow_Tag_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**Importing the required modules**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import random
from bs4 import BeautifulSoup
import time
import warnings; warnings.simplefilter('ignore')
import re
import string

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import ToktokTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import pos_tag

from sklearn.utils import shuffle
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import GaussianNB
import gensim
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


nltk.download('averaged_perceptron_tagger') # required for parts of speech
nltk.download('wordnet') # required for parts of speech
nltk.download('stopwords') #download the stopwords


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

**Web Scraping the current set of questions for testing the model**

In [ ]:


Questions=[] #array to store the questions

for pageNumber in range(1,401):
    response=requests.get("https://stackoverflow.com/questions",params={"tab":"newest","page":pageNumber,"pagesize":50})

    data=BeautifulSoup(response.text,'html.parser' ) #parsing the html text

    req_data=data.find(id="questions")

    new_data=req_data.find_all("h3", class_="s-post-summary--content-title") # the tag that contains the question info



    for element in new_data:
        link=element.a.attrs['href']

        response=requests.get(f"https://stackoverflow.com/{link}")  #fetching the content related to the question

        data_questionwise=BeautifulSoup(response.text,'html.parser')

        question_wise_title=data_questionwise.find("div",id="question-header").h1.a.string #title

        question_wise_desc=data_questionwise.find("div",class_="s-prose js-post-body") #description

        all_paragraphs=question_wise_desc.find_all("p")

        total_description_question_wise=""

        for para in all_paragraphs:
            total_description_question_wise+=para.text

        Final_content=question_wise_title+""+total_description_question_wise  #concatenating the title and description

        tag_question_wise=data_questionwise.find("ul",class_="ml0 list-ls-none js-post-tag-list-wrapper d-inline").li.text #tag

        Questions.append([Final_content,tag_question_wise])

    print(pageNumber)   #checking which page questions have been fetched yet

print(len(Questions))








     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 3.2 MB/s eta 0:00:00


**Sql code to combine the input questions and tags table**

In [ ]:
###################################### Code for inner combine ######################################
df1 = pd.read_csv('Questions.csv', encoding='ISO-8859-1')
df2 = pd.read_csv('Tags.csv', encoding='ISO-8859-1')

# combined dataframe of questiontags
df3 = df1.set_index('Id').join(df2.set_index('Id'))
df3 = df3.reset_index()
df4 = pd.DataFrame(df3.Id.value_counts())
df4 = df4.reset_index()

# df5 is the final combined dataframe of questiontags. Randomly picked datapoints from df3
store = {}
df5 = pd.DataFrame()
n_train = 20000
for i in range(n_train):
    key = 0
    while(key==0):
        a = random.randint(df4.shape[0])
        if a in store:
            continue
        else:
            key==1
            store[a] = 1
            ind = df3[df3.Id==df4.iloc[a,0]].first_valid_index()
            df5 =  pd.concat([df5,pd.DataFrame(df3.iloc[ind]).T],ignore_index=True)
            break

###################################### Code for removing angular brackets from the "Body" column ######################################

for j in range(df5.shape[0]):
    ans=" "
    curr = 0
    flag = 0
    while(curr<len(df5['Body'][j])):
        if( df5['Body'][j][curr]=='<'):
            flag = 1
            curr = curr + 1
            continue
        if( df5['Body'][j][curr]=='>'):
            flag = 0
            curr = curr + 1
            continue
        if(flag == 1):
            curr = curr + 1
        if(flag == 0):
            ans = ans + df5['Body'][j][curr]
            curr = curr + 1
    df1['Title'][j] += ans

df6 = pd.concat([df5.iloc[:,6], df5.iloc[:,8]], axis=1)

**Machine learning part (Preprocessing and training)**

---



In [2]:
train_data=pd.read_csv("questiontags_train.csv")
test_data=pd.read_csv("questiontags_test.csv")

In [3]:
#rename the columns of the train dataset

train_data.drop(train_data.columns[0],axis=1,inplace=True)


train_data.rename(columns={"Title":"Questions","Tag":"Tags"},inplace=True)


In [4]:
import random
#code to include around 10000 more questions to the train dataset from the test dataset
v={}
count=0
while count!=10000:
    idx=random.randint(0,test_data.shape[0]-1)
    if idx not in v:
      train_data=train_data.append(test_data.loc[idx],ignore_index=True)
      test_data.drop(idx,inplace=True,axis=0)
      count+=1
      v[idx]=1
    else:
      continue


In [5]:
train_data['Questions']=train_data['Questions'].str.replace('</p>',' ')
train_data['Questions']=train_data['Questions'].str.replace('\n',' ')
train_data['Questions']=train_data['Questions'].str.replace('</a>',' ')
train_data['Questions']=train_data['Questions'].apply(lambda x : re.sub('(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)' , ' ' , x)) #removing any urls
train_data['Questions'] = train_data['Questions'].apply(lambda x: ' '.join([w for w in x.split() if len(w) > 3])) #removes small length words (len<3)
train_data['Questions']=train_data['Questions'].str.replace('<a href=" ">','')

train_data['Questions']=train_data['Questions'].apply(lambda x : x.lower()) #coverting to lowercase


In [6]:
test_data['Questions']=test_data['Questions'].str.replace('</p>',' ')
test_data['Questions']=test_data['Questions'].str.replace('\n',' ')
test_data['Questions']=test_data['Questions'].str.replace('</a>',' ')
test_data['Questions'] = test_data['Questions'].apply(lambda x: ' '.join([w for w in x.split() if len(w) > 3]))
test_data['Questions']=test_data['Questions'].str.replace('<a href=" ">','')
test_data['Questions']=test_data['Questions'].apply(lambda x : x.lower())
test_data['Questions']=test_data['Questions'].apply(lambda x : re.sub('(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)' , ' ' , x)) #removing any urls

In [7]:
#removing the stop words and the punctuations from test and train dataset


stop_words=set(nltk.corpus.stopwords.words('english'))
train_data['Questions']=train_data['Questions'].apply(lambda x:' '.join([w for w in x.split() if w not in stop_words]))
test_data['Questions']=test_data['Questions'].apply(lambda x:' '.join([w for w in x.split() if w not in stop_words]))

In [8]:
#removing the punctuations from test and train dataset


punctuations = string.punctuation
train_data['Questions']=train_data['Questions'].apply(lambda x:' '.join([''.join([char for char in w if char not in punctuations]) for w in x.split()]))
test_data['Questions']=test_data['Questions'].apply(lambda x:' '.join([''.join([char for char in w if char not in punctuations]) for w in x.split()]))



In [9]:
test_data=test_data.reset_index()


In [10]:

test_data.drop(['index'],inplace=True,axis=1)

In [11]:
#dropping the rows with empty values of Question after filtering
for j in range(len(test_data['Questions'])):
  if len(test_data['Questions'][j])==0:
     test_data.drop(j,inplace=True)

for j in range(len(train_data['Questions'])):
  if len(train_data['Questions'][j])==0:
     train_data.drop(j,inplace=True)
     print("yes")

yes
yes
yes
yes
yes
yes
yes


In [12]:
test_data=test_data.reset_index()
train_data=train_data.reset_index()

In [13]:
test_data.drop(['index'],inplace=True,axis=1)
train_data.drop(['index'],inplace=True,axis=1)


In [14]:
#using lemmatization on the questions of the train and test dataset
def lemmatization(text):
    pos_dict = {
        'N': 'n',  # Noun
        'V': 'v',  # Verb
        'R': 'r',  # Adverb
        'J': 'a'   # Adjective
    }
    pos_tags = pos_tag(text)
    lemmatizer = WordNetLemmatizer()
    lemma=[]
    for word, tag in pos_tags:
        if (tag[0].upper() not in pos_dict.keys()):
          pos='n'
        else:
          pos= pos_dict[tag[0].upper()]
        lemma.append(lemmatizer.lemmatize(word,pos=pos))
    return lemma

train_data['Questions']=train_data['Questions'].apply(lambda x : lemmatization(x.split()))
test_data['Questions']=test_data['Questions'].apply(lambda x : lemmatization(x.split()))
train_data["Questions"]=train_data["Questions"].apply(lambda x : " ".join(x))

test_data["Questions"]=test_data["Questions"].apply(lambda x : " ".join(x))


**Data Exploration with dev of Base Model**

In [15]:
questions = train_data['Questions'].tolist()
tags = train_data['Tags'].tolist()

print('The total number of words in the data is: ', sum([len(text.split()) for text in questions]))

def tokenize_question(text):
    return text.split()

question_vect = CountVectorizer(tokenizer=tokenize_question)
questions=question_vect.fit_transform(questions)

print('The number of words in the vocabulary is: ', len(question_vect.vocabulary_))

def tokenize_tags(text):
    return text.split('|')

max_tags = 100
tags_vect = CountVectorizer(tokenizer=tokenize_tags, max_features=max_tags)
tags = tags_vect.fit_transform(tags)
tags = tags.toarray()
print('Number of tags: ', len(tags_vect.vocabulary_))

tags_token = tags_vect.get_feature_names_out()
tag_frequency = tags.sum(axis=0)
print('The list of tags with frequency is: ')
print(dict(zip(tags_token, tag_frequency)))



The total number of words in the data is:  2199167
The number of words in the vocabulary is:  269417
Number of tags:  100
The list of tags with frequency is: 
{'.htaccess': 31, '.net': 152, 'actionscript-3': 39, 'ajax': 42, 'algorithm': 69, 'amazon-web-services': 121, 'android': 1406, 'angular': 101, 'angularjs': 163, 'apache': 55, 'apache-spark': 31, 'arrays': 54, 'asp.net': 220, 'asp.net-mvc': 99, 'assembly': 28, 'azure': 118, 'bash': 88, 'batch-file': 29, 'c': 401, 'c#': 1982, 'c++': 991, 'cakephp': 28, 'css': 259, 'database': 85, 'delphi': 55, 'django': 152, 'docker': 101, 'eclipse': 67, 'elasticsearch': 33, 'excel': 214, 'facebook': 55, 'firebase': 33, 'flutter': 206, 'git': 149, 'go': 67, 'google-apps-script': 38, 'google-cloud-platform': 27, 'hadoop': 42, 'haskell': 37, 'hibernate': 26, 'html': 488, 'html5': 39, 'image': 28, 'ios': 686, 'iphone': 278, 'java': 2322, 'javascript': 2893, 'jquery': 528, 'json': 92, 'kubernetes': 53, 'laravel': 54, 'linux': 198, 'machine-learning': 2

In [16]:
#checking the top most tags based on the frequency

tag_count = [[x[0],x[1]] for x in zip(tags_token, tag_frequency)]
tag_count = sorted(tag_count, key = lambda x: x[1], reverse=True)

print('Rank     Tag      Count')
for i in range(100):
    print("%d   %s   %d" % (i+1, tag_count[i][0], tag_count[i][1]))

Rank     Tag      Count
1   javascript   2893
2   java   2322
3   python   2285
4   c#   1982
5   php   1727
6   android   1406
7   c++   991
8   ios   686
9   jquery   528
10   html   488
11   r   478
12   sql   472
13   c   401
14   ruby-on-rails   379
15   mysql   370
16   node.js   328
17   reactjs   302
18   iphone   278
19   css   259
20   asp.net   220
21   excel   214
22   flutter   206
23   linux   198
24   objective-c   171
25   angularjs   163
26   .net   152
27   django   152
28   git   149
29   ruby   145
30   sql-server   140
31   windows   139
32   amazon-web-services   121
33   swift   119
34   azure   118
35   regex   106
36   typescript   103
37   wordpress   103
38   angular   101
39   docker   101
40   python-3.x   100
41   asp.net-mvc   99
42   xml   96
43   json   92
44   vb.net   92
45   postgresql   90
46   matlab   89
47   bash   88
48   database   85
49   scala   82
50   spring   81
51   wpf   80
52   powershell   72
53   mongodb   70
54   algorithm   69
55   

In [17]:
# X_test = test_data['Questions'].tolist()
# y_test = test_data['Tags'].tolist()

In [18]:
# X_train, X_test, y_train, y_test = train_test_split(questions, tags,
#                                                     test_size=0.2,
#                                                     random_state=747)

# X_val, X_test, y_val, y_test = train_test_split(X_test, y_test,
#                                                     test_size=0.5,
#                                                     random_state=747)

In [19]:
#function to calculate the evaluation metrics

def eval_metrics(y_test, y_predicted, print_metrics=True):

    accuracy = accuracy_score(y_test, y_predicted)
    precision = precision_score(y_test, y_predicted, average='weighted')
    recall = recall_score(y_test, y_predicted, average='weighted')
    f1 = f1_score(y_test, y_predicted, average='weighted')

    if print_metrics:
        print("f1: %.3f - precision: %.3f - recall: %.3f - accuracy: %.3f" % (
            f1, precision, recall, accuracy))
    return f1, precision, recall, accuracy

In [32]:
train_data.head()

train_new=train_data.copy()
test_new=test_data.copy()

In [21]:
# test_new.loc[0,"Questions"]="hi this is my area"


In [34]:


def tokenize_question(text):
    return text.split()

tfidf_vect = TfidfVectorizer(tokenizer=tokenize_question,
                               stop_words='english',
                               min_df=4,
                               max_df=0.5,max_features=7000)

X_train_tfidf = tfidf_vect.fit_transform(train_new["Questions"]).todense()
X_test_tfidf = tfidf_vect.transform(test_new["Questions"]).todense()
# print('The number of words in the vocabulary is: ', len(tfidf_vect.vocabulary_))

def filter_number_features(name):
  if(name[0] in ('0123456789') and len(name)<4):
    return False
  return True

#get the feature names
feature_names=tfidf_vect.get_feature_names_out()

# Get the IDF scores
idf_scores = tfidf_vect.idf_

Final_Feature_Set=[]
for idx,feature_name in enumerate(feature_names):
    if filter_number_features(feature_name):
       Final_Feature_Set.append([feature_name,idf_scores[idx]])

Final_Feature_Set=sorted(Final_Feature_Set,key=lambda x :x[1],reverse=True)



Streaming output truncated to the last 5000 lines.
clientsecret 8.536163918401844
colorwhite 8.536163918401844
column2 8.536163918401844
comp 8.536163918401844
companyid 8.536163918401844
connectionstring 8.536163918401844
conopen 8.536163918401844
consumes 8.536163918401844
contextmenu 8.536163918401844
coredata 8.536163918401844
corporation 8.536163918401844
cucumber 8.536163918401844
curlinit 8.536163918401844
cypress 8.536163918401844
databinding 8.536163918401844
datatemplate 8.536163918401844
deviation 8.536163918401844
dial 8.536163918401844
dialoginterfaceonclicklistener 8.536163918401844
dialogue 8.536163918401844
dictionaryltstring 8.536163918401844
dirs 8.536163918401844
downloaded 8.536163918401844
endregion 8.536163918401844
exmessage 8.536163918401844
experimental 8.536163918401844
fileinputstream 8.536163918401844
finder 8.536163918401844
flicker 8.536163918401844
fontweightbold 8.536163918401844
food 8.536163918401844
functiond 8.536163918401844
generalize 8.53616391840

,Questions,Tags
0,hi this is my area,go
1,list user define table specific database sybas...,java
2,ruby rail common method available controller v...,ruby-on-rails
3,vectorizing loop greeting vectorizing loop tre...,matlab
4,move expressjs apps electronjs apps try unders...,node.js


In [ ]:
%%time
tfidf_log_clf = OneVsRestClassifier(LogisticRegression())

tfidf_log_clf.fit(X_train_tfidf, y_train)

CPU times: user 27.6 s, sys: 28.5 s, total: 56.1 s
Wall time: 32 s


OneVsRestClassifier(estimator=LogisticRegression())

In [ ]:
y_train_tfidf_predict = tfidf_log_clf.predict(X_train_tfidf)
eval_metrics(y_train, y_train_tfidf_predict)

f1: 0.287 - precision: 0.705 - recall: 0.192 - accuracy: 0.325


(0.2869979496204687,
 0.7048346764490622,
 0.19239736441966548,
 0.3247759949989581)

In [ ]:
y_test_tfidf_predict = tfidf_log_clf.predict(X_test_tfidf)
eval_metrics(y_test, y_test_tfidf_predict)

f1: 0.246 - precision: 0.563 - recall: 0.164 - accuracy: 0.287


(0.24568413711051668, 0.5629343082208758, 0.1643342662934826, 0.287)